<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# Data-Driven Finance

**Illustrated based on Different Types of Financial Data**

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

<p style='color: red;'><b>This notebook requires access to a number of APIs based on individual credentials; some of the APIs do not even work anymore the way they are used in the notebook. The notebook can only be used for illustation.</p></b>

## Imports

In [ ]:
!git clone https://github.com/tpq-classes/ai_in_finance.git
import sys
sys.path.append('ai_in_finance')


In [ ]:
!pip install git+https://github.com/yhilpisch/tpqoa

In [ ]:
from fix_cufflink import *
cf_colors.to_rgba = fixed_to_rgba
cf_plotlytools.to_rgba = fixed_to_rgba

In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt
import plotly.io as pio
plt.style.use('seaborn-v0_8')
import cufflinks
cufflinks.set_config_file(offline=True)
pio.renderers.default = "colab"

In [ ]:
import warnings as w; w.simplefilter('ignore')

## Scientific Method

### Problem

What is the standard gravity value $g$ on earth (earth-induced acceleration of an object towards the ground caused by gravity)?

See eg https://en.wikipedia.org/wiki/Gravity.

### Theory

The speed of an object is given by

$$v = a \cdot t$$

with $v$ the speed, $a$ acceleration and $t$ the time for which acceleration takes place.

The distance  $d$ an object covers over a period of $t$ with speed $v$ is

$$d = v \cdot t$$

One also gets

$$d = a \cdot t^2$$

or

$$a = \frac{d}{t^2}$$

### Experiment

We let an object drop to the ground from different heights $h$ and measure the times $t$ it takes until the object reaches the ground.

Hypothesis: the gravity value $g$ is dependent of the height.

### Measurements

In [ ]:
h = np.array([1, 1.5, 2, 2.5, 3, 3.5, 4])

We simulate experimental data through the relationship

$$t = \sqrt{\frac{h}{g}} + \epsilon = \sqrt{\frac{h}{9.80665}} + \epsilon$$

where $\epsilon$ represents zero-mean (measurement) noise.

In [ ]:
t = np.sqrt(h / 9.80665) + np.random.standard_normal(len(h)) / 5000
t

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(h, t, 'ro')
plt.xlabel('initial height $h$')
plt.ylabel('time to ground $t$');

At first inspection, the relationship between $t$ and $h$ seems linear.

However, adding observations with larger values for $h$ shows that it is not linear.

In [ ]:
h = np.array([0.01, 1, 1.5, 2, 2.5, 3, 3.5, 4, 8, 16, 32, 64])
t = np.sqrt(h / 9.80665) + np.random.standard_normal(len(h)) / 5000

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(h, t, 'ro')
plt.xlabel('initial height $h$')
plt.ylabel('time to ground $t$');

Given the measurements (the data), a new hypothesis can be formulated:

$$t = c \cdot \sqrt{h}$$

### First Results

Plugging in the data to the formula $c^2 = \frac{h}{t^2}$ gives

In [ ]:
c2 = h / t ** 2
c2.round(5)

We can identify $c^2$ with $g$, the gravity value.

All these values are pretty close to each other.

Taking the average already gives a good estimate for (the real) $g$.

In [ ]:
g = c2
g.mean()

### More Data

Let us do more experiements. This time the height is fixed to $h=1$ meter.

In [ ]:
h = np.array(1000 * [1,])
t = np.sqrt(h / 9.80665) + np.random.standard_normal(len(h)) / 5000

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(h, t, 'ro')
plt.xlabel('initial height $h$')
plt.ylabel('time to ground $t$');

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(t, bins=35)
plt.ylabel('frequency')
plt.xlabel('time to ground $t$');

With more data, the estimate is more accurate (given simulated measurement zero-mean noise).

In [ ]:
g = h / t ** 2
g.mean()

## Structured Data

### EOD Data

In [ ]:
url = 'http://hilpisch.com/tr_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
raw.info()

In [ ]:
symbols = ['AAPL.O', 'MSFT.O', 'INTC.O']

In [ ]:
data = raw[symbols].copy()

In [ ]:
rets = np.log(data / data.shift(1)).dropna()

In [ ]:
data.normalize().iplot()

### Intraday Data

See https://github.com/yhilpisch/tpqoa.

In [ ]:
import tpqoa

In [ ]:
oanda = tpqoa.tpqoa('/notebooks/data/pyalgo.cfg')

In [ ]:
symbol = 'EUR_USD'

In [ ]:
raw = oanda.get_history(symbol, '2019-02-04', '2019-02-06', 'M1', 'A')

In [ ]:
raw.info()

In [ ]:
raw['c'].iplot()

### Tick Data &mdash; Historical

See https://fxcmpy.tpq.io.

In [ ]:
import fxcmpy
import datetime as dt
from fxcmpy import fxcmpy_tick_data_reader as tdr

In [ ]:
fxcm = fxcmpy.fxcmpy(config_file='/notebooks/data/pyalgo.cfg')

In [ ]:
data = fxcm.get_candles('EUR/USD', period='m1', number=250)

In [ ]:
(data['askclose'] - data['bidclose']).mean()

In [ ]:
data[['askclose', 'bidclose']].iplot()

In [ ]:
start = dt.datetime(2018, 12, 1)
end = dt.datetime(2018, 12, 7)

In [ ]:
tick = tdr('EURUSD', start, end, verbosity=True)

In [ ]:
tick.get_raw_data().info()

In [ ]:
%time ticks = tick.get_data()

In [ ]:
ticks.iloc[::100].iplot()

### Tick Data &mdash; Streaming

In [ ]:
oanda.stream_data('EUR_USD', stop=10)

## Unstructured Data

### Texts

In [ ]:
!apt install poppler-utils

In [ ]:
!pip install lxml[html_clean]

`brew cask install pdftotext`

In [ ]:
!pdftotext /content/ai_in_finance/Walden-by-Henry-David-Thoreau.pdf /content/ai_in_finance/Walden-by-Henry-David-Thoreau.txt

In [ ]:
ls /content/ai_in_finance/Wa*

In [ ]:
fn = '/content/ai_in_finance/Walden-by-Henry-David-Thoreau.txt'

In [ ]:
!head -n 20 $fn

In [ ]:
%time f = open(fn, 'r').read()

In [ ]:
f.split('\n')[40]

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

In [ ]:
import nlp_functions as nlp

In [ ]:

nlp.generate_word_cloud(f, 40)

### Dow Jones DNA

In [ ]:
musk = pd.read_hdf('/content/ai_in_finance/snapshot_musk_100.h5')

In [ ]:
musk.info()

In [ ]:
musk[['title', 'publisher_name', 'publication_date']].head()

In [ ]:
musk['body'].head()

## Alternative Data

### Web

In [ ]:
import requests

In [ ]:
sources = [
    'https://nr.apple.com/dE0b1T5G3u',  # iPad Pro
    'https://nr.apple.com/dE4c7T6g1K',  # MacBook Air
    'https://nr.apple.com/dE4q4r8A2A',  # Mac Mini
]

In [ ]:
html = [requests.get(url).text for url in sources]

In [ ]:
data = [nlp.clean_up_text(t) for t in html]

In [ ]:
data[0][535:1000]

### Twitter

Developer documentation for Twitter: https://developer.twitter.com/en/docs

In [ ]:
import configparser as cp
from twitter import Twitter, OAuth

In [ ]:
cfg = cp.ConfigParser()
cfg.read('dyjh.cfg')

In [ ]:
t = Twitter(auth=OAuth(cfg['twitter']['access_token'],
                       cfg['twitter']['access_secret_token'],
                       cfg['twitter']['api_key'],
                       cfg['twitter']['api_secret_key']),
           retry=True)

In [ ]:
l = t.statuses.home_timeline(count=5)

In [ ]:
for e in l:
    print(e['text'])

In [ ]:
l = t.statuses.user_timeline(screen_name='dyjh', count=5)

In [ ]:
for e in l:
    print(e['text'])

In [ ]:
d = t.search.tweets(q='#Python', count=5)

In [ ]:
for e in d['statuses']:
    print(e['text'])

In [ ]:
l = t.statuses.user_timeline(screen_name='realDonaldTrump', count=75)

In [ ]:
tl = [e['text'] for e in l]

In [ ]:
tl[-5:]

In [ ]:
nlp.generate_word_cloud(' '.join(tl), 35)

## Eikon

### Price Data

In [ ]:
import eikon as ek

In [ ]:
ek.set_app_key(cfg['eikon']['app_id'])

In [ ]:
rics = [
    'GE',  # General Electric stock
    'AAPL.O',  # Apple stock
    '.SPX',  # S&P 500 stock index
    '.VIX',  # VIX volatility index
    'EUR=',  # EUR/USD exchange rate
    'XAU=',  # Gold price
    'DE10YT=RR',  # 10yr Bund price
]

In [ ]:
data = ek.get_timeseries(rics,  # the RICs
                         fields='CLOSE',  # the required fields
                         start_date='2018-01-01',  # start date
                         end_date='2019-02-08')  # end date

In [ ]:
data.head()  # first five rows

In [ ]:
data = ek.get_timeseries(rics,  # RICs
              fields='CLOSE',  # fields to be retrieved
              start_date='2020-02-07 14:00:00',  # start time
              end_date='2020-02-07 18:00:00',  # end time
              interval='minute')  # bar length

In [ ]:
data.head()

In [ ]:
data.normalize().iplot()

### Fundamentals

In [ ]:
data_grid, err = ek.get_data(['AAPL.O', 'IBM', 'GOOG.O', 'MSFT.O'],
        ['TR.TotalReturnYTD', 'TR.WACCBeta', 'YRHIGH', 'YRLOW', 'TR.GrossProfit'])

In [ ]:
data_grid

In [ ]:
data_grid.set_index('Instrument')[['YTD Total Return', 'Beta']].iplot(
                    kind='bar', subplots=True)

In [ ]:
data_grid.set_index('Instrument')[['YRHIGH', 'YRLOW']].iplot(kind='bar')

In [ ]:
data_grid.set_index('Instrument')['Gross Profit'].iplot(kind='bar')

### News

In [ ]:
news = ek.get_news_headlines('R:TSLA.O PRODUCTION',
                         date_from='2020-05-01',
                         date_to='2020-05-08',
                         count=5
                        )

In [ ]:
news

In [ ]:
storyId = news['storyId'][0]

In [ ]:
from IPython.display import HTML

In [ ]:
HTML(ek.get_news_story(storyId))

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">